In [ ]:
import IPython.display as ipd

import torch

from model import commons
from params import Params
from text.convert import text_to_sequence
from model.synthesizer import SynthesizerTrn
from utils.checkpoint import load_checkpoint

In [ ]:
def get_text(text: str, text_cleaners: list[str], language: str):
    text_norm = text_to_sequence(text, text_cleaners, language)
    text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)

    return text_norm

In [ ]:
params = Params.parse_file('files/configs/lt.json')

net_g = SynthesizerTrn.from_params(params)
_ = net_g.eval()

In [ ]:
load_checkpoint('G_105000.pth', net_g, None)

In [ ]:
text = 'Penktãdienio nãktį daũg kur̃ trumpaĩ palìs'
# text = 'Gramãtiškai veiksmãžodis dẽrinamas sù veiksniù'
# text = 'màno var̃das Gãbija'
# text = 'Ar̃ nóri eĩti sù manimì draugè'

stn_tst = get_text(text, params.data.text_cleaners, params.data.language)

with torch.inference_mode():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=0.667, noise_scale_w=0.5, length_scale=1.0)[0][0,0]

ipd.display(ipd.Audio(audio.data.float().numpy(), rate=params.data.sampling_rate, normalize=False))